# **Projeto KNN**
O algoritmo K-Nearest Neighbors (KNN) é uma técnica de aprendizado supervisionado usada tanto para classificação quanto para regressão. Ele funciona baseado na premissa de que dados similares estão próximos uns dos outros em um espaço de características.


### **1. Problema de Negócio**
Construir uma machine learning para avaliar o dano de um veiculo e classifica-lo em alto, médio ou baixo.

In [21]:
# Carregando os dados
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder # transforma variaveis categóricas em numericas
from ipywidgets import widgets, interactive #para criar o grafico
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Importando bibliotecas de visualização
import ipywidgets as widgets
from IPython.display import display

style = {'Descrição_Variáveis': 'initial'}

### **2. Análise Exploratória**

In [4]:
#Importando arquivo de dados

dados = pd.read_csv('/content/dano.csv', delimiter=';')

dados.head()

,ID_Veicular,Laterais,Dianteira,Traseira,Teto,Dano_Veicular
0,0,5.1,3.5,1.4,0.2,Dano-Alto
1,1,4.9,3.0,1.4,0.2,Dano-Alto
2,2,4.7,3.2,1.3,0.2,Dano-Alto
3,3,4.6,3.1,1.5,0.2,Dano-Alto
4,4,5.0,3.6,1.4,0.2,Dano-Alto



**Dicionário de dados**
- ID_Veicular - Número de identificaçã do veículo.
- Laterais - Nota de avaliação das laterais do carro definida pela area de negócio.
- Dianteira - Nota de avaliação da dianteira do carro definida pela area de negócio.
- Traseia - Nota de avaliação da traseira do carro definida pela area de negócio.
- Teto - Nota de avaliação do teto do carro definida pela area de negócio.
- Dano-veicular - Definição do nível do dano que pode ser classificado em alto, médio ou baixo.
Observação: a nota de avaliação pode ser um valor atribuido por uma ML através da análise de fotos do veículo.


In [5]:
#Outra forma para verificar quantas colunas e registro o arquivo possui
print(f"Dataset tem {dados.shape[0]} linhas e {dados.shape[1]} colunas.")

Dataset tem 150 linhas e 6 colunas.


In [6]:
# Dropdown para seleção de coluna
wig_col = widgets.Dropdown(
    options=[col for col in dados.columns if col.startswith(('Laterais', 'Dianteira', 'Traseira', 'Teto'))],
    description='Coluna:',
    disabled=False,
    layout=widgets.Layout(width='30%')
)

# Função para criar o gráfico
def plot_graph(coluna):
    sns.catplot(
        x="Dano_Veicular",
        y=coluna,
        kind="boxen",
        data=dados,
        height=6,
        aspect=1
    )
    plt.show()

# Conexão do dropdown com a função
interactive_plot = interactive(plot_graph, coluna=wig_col)
display(wig_col, interactive_plot)


Dropdown(description='Coluna:', layout=Layout(width='30%'), options=('Laterais', 'Dianteira', 'Traseira', 'Tet…

interactive(children=(Dropdown(description='Coluna:', layout=Layout(width='30%'), options=('Laterais', 'Diante…

Esse grafico permite avaliar como e o comportamento do nivel do dano isolando cada uma das variaveis (lateral, dianteira, traseira e teto).
Exemplo: quando o dano na lateral podemos avaliar e a nota para dano alto fica entre 2 e 5.8 mais ou menos. Assim podemos avliar como as variaveis se comportam isoladamente para classificação do dano.

### **3. Pré processamento dos dados**

In [7]:
# Separando as Variáveis de Entrada e Saída do Modelo
X = dados.iloc[:,:-1].values  #variáveis independentes
y = dados.iloc[:,-1].values  #Target/labe/class/vairável dependente

In [8]:
# Conferindo a distribuição
print("\nVariáveis de Entrada da avaliação:\n\n", X[:5])
print("\nResultado do Dano :\n\n", y[:5])


Variáveis de Entrada da avaliação:

 [[0.  5.1 3.5 1.4 0.2]
 [1.  4.9 3.  1.4 0.2]
 [2.  4.7 3.2 1.3 0.2]
 [3.  4.6 3.1 1.5 0.2]
 [4.  5.  3.6 1.4 0.2]]

Resultado do Dano :

 ['Dano-Alto' 'Dano-Alto' 'Dano-Alto' 'Dano-Alto' 'Dano-Alto']


In [9]:
dados.head()

,ID_Veicular,Laterais,Dianteira,Traseira,Teto,Dano_Veicular
0,0,5.1,3.5,1.4,0.2,Dano-Alto
1,1,4.9,3.0,1.4,0.2,Dano-Alto
2,2,4.7,3.2,1.3,0.2,Dano-Alto
3,3,4.6,3.1,1.5,0.2,Dano-Alto
4,4,5.0,3.6,1.4,0.2,Dano-Alto


### Separando os dados entre Treino e Teste de forma interativa

In [10]:
# Criando o dataser de treino e de teste de forma iteratira
from sklearn.model_selection import train_test_split

test_size = widgets.FloatSlider(min=0.01, max=0.6, value=0.2, description="Test Size :", tooltips=['Usually 20-30%'])
display(test_size)

FloatSlider(value=0.2, description='Test Size :', max=0.6, min=0.01)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X ,y, test_size=test_size.value, random_state = 7)

In [12]:
print("Dados de Treino :\n----------------\n")
print("X = \n", X_train[:2])
print("y = \n", y_train[:2])

print("\n\nDados de Teste :\n----------------\n")
print("X = \n",X_test[:2])
print("y = \n", y_test[:2])

Dados de Treino :
----------------

X = 
 [[126.    6.2   2.8   4.8   1.8]
 [ 79.    5.7   2.6   3.5   1. ]]
y = 
 ['Dano-Baixo' 'Dano-Medio']


Dados de Teste :
----------------

X = 
 [[149.    5.9   3.    5.1   1.8]
 [ 84.    5.4   3.    4.5   1.5]]
y = 
 ['Dano-Baixo' 'Dano-Medio']


In [13]:
# Verificando a quantidade de registros no dataset de dados e no de treino
print(f"Shape dos dados de treino é {X_train.shape}")
print(f"Shape dos dados de teste é {X_test.shape}")

Shape dos dados de treino é (120, 5)
Shape dos dados de teste é (30, 5)


### Padronizando as variáveis porque elas estão em escalas diferentes.

In [14]:
dados.describe()

,ID_Veicular,Laterais,Dianteira,Traseira,Teto
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,74.500000,5.843333,3.054000,3.758667,1.198667
std,43.445368,0.828066,0.433594,1.764420,0.763161
min,0.000000,4.300000,2.000000,1.000000,0.100000
25%,37.250000,5.100000,2.800000,1.600000,0.300000
50%,74.500000,5.800000,3.000000,4.350000,1.300000
75%,111.750000,6.400000,3.300000,5.100000,1.800000
max,149.000000,7.900000,4.400000,6.900000,2.500000


In [15]:
# Padronizando as variáveis
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train = sc.fit_transform(X_train) #Executa a padronização e sobrescreve os dados na tabela
X_test = sc.transform(X_test)

print("\n-------------------------\nDataset depois da padronização:\n-------------------------\n", )

print("\nX_train :\n", X_train[:5])
print("-------------------------")
print("\nX_test :\n", X_test[:5])


-------------------------
Dataset depois da padronização:
-------------------------


X_train :
 [[ 1.20732325  0.43325894 -0.59653573  0.62601116  0.82081788]
 [ 0.14948526 -0.15754871 -1.05688485 -0.09737951 -0.21464226]
 [-1.13342464 -1.45732553  1.24486074 -1.48851543 -1.2501024 ]
 [ 1.49991673  1.26038965  0.09398794  0.95988378  1.20911544]
 [ 0.03694931  0.669582   -0.36636117  0.34778398  0.17365529]]
-------------------------

X_test :
 [[ 1.72498864  0.07877435 -0.13618661  0.79294747  0.82081788]
 [ 0.26202122 -0.51203329 -0.13618661  0.45907485  0.43252033]
 [-0.72829519 -0.98467941  1.01468618 -1.32157912 -1.12066988]
 [-0.14310822 -0.27571024 -0.13618661  0.45907485  0.43252033]
 [ 0.75717942 -1.10284094 -1.28705941  0.45907485  0.69138537]]


### **4. Criação da Máquina Preditiva**

In [16]:
# Importa a biblioteca KNN
from sklearn.neighbors import KNeighborsClassifier

# Configura os parâmetros do modelo e permite selecionar o K
neigh_wig = widgets.Dropdown(options=[1, 10, 30, 100],
                             description="Number of Neighbours = ", style=style)

display(neigh_wig)

Dropdown(description='Number of Neighbours = ', options=(1, 10, 30, 100), value=1)

In [28]:
# Construção do Máquina preditiva
modelo = KNeighborsClassifier(n_neighbors=neigh_wig.value)

modelo = modelo.fit(X_train,y_train)

#Fazendo a predição do modelo
y_pred = modelo.predict(X_test)

print("\n---------------------------\n")
print("Valores Preditos com dados de Teste :\n",y_pred)
print("\n---------------------------\n")
print("Valores Reais com dados de teste :\n",y_test)


---------------------------

Valores Preditos com dados de Teste :
 ['Dano-Baixo' 'Dano-Medio' 'Dano-Alto' 'Dano-Medio' 'Dano-Medio'
 'Dano-Alto' 'Dano-Medio' 'Dano-Medio' 'Dano-Alto' 'Dano-Medio'
 'Dano-Medio' 'Dano-Medio' 'Dano-Alto' 'Dano-Baixo' 'Dano-Alto'
 'Dano-Medio' 'Dano-Baixo' 'Dano-Baixo' 'Dano-Alto' 'Dano-Alto'
 'Dano-Medio' 'Dano-Baixo' 'Dano-Medio' 'Dano-Baixo' 'Dano-Baixo'
 'Dano-Baixo' 'Dano-Medio' 'Dano-Medio' 'Dano-Baixo' 'Dano-Baixo']

---------------------------

Valores Reais com dados de teste :
 ['Dano-Baixo' 'Dano-Medio' 'Dano-Alto' 'Dano-Medio' 'Dano-Baixo'
 'Dano-Alto' 'Dano-Medio' 'Dano-Medio' 'Dano-Alto' 'Dano-Medio'
 'Dano-Medio' 'Dano-Medio' 'Dano-Alto' 'Dano-Baixo' 'Dano-Alto'
 'Dano-Medio' 'Dano-Baixo' 'Dano-Baixo' 'Dano-Alto' 'Dano-Alto'
 'Dano-Medio' 'Dano-Baixo' 'Dano-Medio' 'Dano-Baixo' 'Dano-Baixo'
 'Dano-Baixo' 'Dano-Medio' 'Dano-Medio' 'Dano-Baixo' 'Dano-Baixo']


### **5. Avalindo a Máquina Preditiva**

In [29]:
#Avaliando a máquina preditiva
from sklearn import metrics
print('Acurácia: %.2f'% (accuracy_score(y_test, y_pred)*100), '%')
#print('Acurácia: %.2f'% (modelo.score(X_test, y_test)*100), '%' ) 2 formas para calcular a acurácia do modelo
# Acurácia: 90.00 % Com um vizinho
# Acurácia: 96.67 % Com 10 vizinhos  MELHOR RESULTADO!
# Acurácia: 93.33 % Com 30 vizinhos
# Acurácia: 76.67 % Com 100 vizinhos

Acurácia: 96.67 %


In [31]:
#9 Comparando Real e Predito com dados de Teste
print("\nReal vs Predito \n------------------------------\n")
diferenca = pd.DataFrame({"Real" : y_test,
                         "Predito" : y_pred})

diferenca


Real vs Predito 
------------------------------



,Real,Predito
0,Dano-Baixo,Dano-Baixo
1,Dano-Medio,Dano-Medio
2,Dano-Alto,Dano-Alto
3,Dano-Medio,Dano-Medio
4,Dano-Baixo,Dano-Medio
5,Dano-Alto,Dano-Alto
6,Dano-Medio,Dano-Medio
7,Dano-Medio,Dano-Medio
8,Dano-Alto,Dano-Alto
9,Dano-Medio,Dano-Medio


In [32]:
#Usando a confusion matrix para avaliar a acurácia e demais indicadores

cm = confusion_matrix(y_test,y_pred)
cr = classification_report(y_test, y_pred)
print(cm)
print(cr)

[[ 7  0  0]
 [ 0 10  1]
 [ 0  0 12]]
              precision    recall  f1-score   support

   Dano-Alto       1.00      1.00      1.00         7
  Dano-Baixo       1.00      0.91      0.95        11
  Dano-Medio       0.92      1.00      0.96        12

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30



Avaliação da matriz de confusão: Em somente um caso que era dano medio a maquina preditiva avaliou como dano médio.
O modelo apresenta expressivo numero de verdadeiros positivo e verdadeiro falso.
Equilíbrio nas métricas de precisão, recall e pontuação F1, e resultados praticamente perfeitos para a classe Dano-Alto.

**A MELHOR MAQUINA PREDITIVA FOI A KNN COM AVALIAÇÃO DOS 10 VIZINHOS MAIS PRÓXIMOS. COMO A ACURÁCIA FOI DE 96.67 % NÃO NECESSITA A AVALIAÇÃO DE OUTRO MODELO PARA COMPARAR.  **